## Start

In [9]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Source #Mobile

### Links: 
___

In [ ]:
# creating a table from s3

query_text = """--sql
    CREATE TABLE db1.flats_st_partner
    (
    `report_date` Date,
    `flat_uuid` String,
    `call_blocked` Int16,
    `blocked` Int16,
    `deleted` Int16
    )
    ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/flats_st_partner/year=*/month=*/*.csv','CSVWithNames')
    """

ch.query_run(query_text)

In [5]:
query_text = """--sql
   CREATE TABLE db1.flats_st_partner_ch
    (
    `report_date` Date,
    `flat_uuid` String,
    `call_blocked` Int16,
    `blocked` Int16,
    `deleted` Int16
    )
    ENGINE = MergeTree()
    ORDER BY flat_uuid
    """

ch.query_run(query_text)

""


In [6]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.flats_st_partner_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.flats_st_partner_ch AS
    SELECT
        *
    FROM db1.flats_st_partner
    """

ch.query_run(query_text)

""


___
## Tools
___
### query


In [10]:
query_text = """--sql
    SELECT
        *
    FROM db1.flats_st_partner_ch
    ORDER BY report_date desc
    limit 100
    """

ch.query_run(query_text)


,report_date,flat_uuid,call_blocked,blocked,deleted
0,2025-09-15,02bfd984-e24d-4355-b89c-0fba900b82a0,0,0,0
1,2025-09-15,02bfdb11-6a51-4072-b9ec-981d167f4c2f,0,0,0
2,2025-09-15,02c00a4c-e5b0-40ca-adf0-31c172241acc,0,0,0
3,2025-09-15,02c01167-8611-4688-8168-a3736bcfd1fa,0,0,0
4,2025-09-15,02c01385-d3a5-464e-ba10-364200e80f80,0,0,0
5,2025-09-15,02c01546-abd1-46d8-8e7f-45d46f3046e0,0,0,0
6,2025-09-15,02c0295d-4236-47dd-b892-e89f63b57f1b,0,0,0
7,2025-09-15,02c064e6-679f-4bd0-ac74-2c63515f40cd,0,0,0
8,2025-09-15,02c065aa-8591-4407-b55f-7d79ea302d98,1,1,0
9,2025-09-15,02c08db2-98f9-40a9-938c-ef6895a3a582,0,0,0


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.flats_st_partner_ch DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


shape: (0, 0)
┌┐
╞╡
└┘

### drop mv

In [8]:
query_text = """--sql
    DROP TABLE db1.flats_st_partner_mv
    """

ch.query_run(query_text)

""


### drop ch

In [2]:
query_text = """--sql
    DROP TABLE db1.flats_st_partner_ch
    """

ch.query_run(query_text)

""


### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.flats_st_partner_mv
"""

ch.query_run(query_text)